# Song Generator

Idea: use the chord progressions thing to generate sequence, then a melody over that sequence that fits according to the number of beats in an input poem

![](poss_chords_prog.png)

---

### Structure
1. Generate chord sequence
2. Do this for verses, chorus, and bridge
3. Parse input test, find number of syllables per line
4. Generate melody with same number of notes over the chord progression
5. (maybe) export to midi files
6. Put into ableton
7. Record vocals
8. Repeat over and over and over...
9. Profit

### SongGen stuff
---

In [2]:
import random
from midiutil.MidiFile import MIDIFile
from datetime import datetime

In [52]:
class SongGeneration:
    
    def __init__(self, key, length=4, verses=1, rhythm=[], file_name="test", file_location=""):
        self.key = key
        self.length = length # default is 4 chords
        self.verses = verses # default is 1 verse
        # absolute maximum recommended length: 9
        self.file_name = file_name
        self.file_location = file_location
        
        # Chord Progression Dictionary
        self.chord_prog_maj = {'I':[['V', 'vii'], ['ii', 'IV'], ['vi'], ['iii']],
                          'ii':['V', 'vii'],
                          'iii': ['vi'],
                          'IV':['V', 'vii'],
                          'V': ['iii'],
                          'vi': ['ii', 'IV'],
                          'vii': ['iii']}
        
        # List of all possible notes and rhythms, plus the weights for picking the rhythm
        self.all_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        self.all_rhythms = [0.25, 0.5, 1, 2]
        self.all_rhythm_weights = [0.1, 0.4, 0.95, 1.0]
        
        # Notes dictionary for MIDI file
        self.notes_dict = {}
        bass = 48
        treble1 = 60
        treble2 = 72
        for i in range(0, len(self.all_notes)):
            self.notes_dict[self.all_notes[i]] = [bass + i, treble1 + i, treble2 + i]
        
        # Class variables, for use in generation functions
        self.scale = []
        self.chords = []
        self.chord_notes = []
        self.melody = []
        self.rhythm_intro = rhythm
        self.rhythm = []
    
    def __str__(self):
        return_string = str(self.scale) + "\n" + str(self.chords) + "\n" 
        #return_string += str(self.chord_notes) + "\n" + str(self.rhythm) + "\n"
        #return_string += str(self.melody)
        return return_string
    
    def gen_song(self):
        self.gen_scale()
        self.gen_chords()
        for i in range(self.verses):
            self.gen_rhythm()
            self.gen_melody()
        self.gen_MIDI()
        
    def gen_scale(self):
        i = self.all_notes.index(self.key)
        l = len(self.all_notes)
        self.scale.append(self.all_notes[i])
        self.scale.append(self.all_notes[(i + 2) % l])
        self.scale.append(self.all_notes[(i + 4) % l])
        self.scale.append(self.all_notes[(i + 5) % l])
        self.scale.append(self.all_notes[(i + 7) % l])
        self.scale.append(self.all_notes[(i + 9) % l])
        self.scale.append(self.all_notes[(i + 11) % l])
        
    def gen_chords(self):
        chords = ['I']
        sub_value = 2
        start_value = (self.length - sub_value) % 4
        start_chord = random.choice(self.chord_prog_maj[chords[0]][start_value])
        chords.append(start_chord)
        
        while len(chords) != self.length:
            next_chord = random.choice(self.chord_prog_maj[chords[-1]])            
            chords.append(next_chord)
            
        self.chords = chords
        
        for chord in self.chords:
            if chord == 'I':
                self.chord_notes.append([self.scale[0], self.scale[2], self.scale[4]])
            elif chord == 'ii':
                self.chord_notes.append([self.scale[1], self.scale[3], self.scale[5]])
            elif chord == 'iii':
                self.chord_notes.append([self.scale[2], self.scale[4], self.scale[6]])
            elif chord == 'IV':
                self.chord_notes.append([self.scale[3], self.scale[5], self.scale[0]])
            elif chord == 'V':
                self.chord_notes.append([self.scale[4], self.scale[6], self.scale[1]])
            elif chord == 'vi':
                self.chord_notes.append([self.scale[5], self.scale[0], self.scale[2]])
            elif chord == 'vii':
                self.chord_notes.append([self.scale[6], self.scale[1], self.scale[3]])
        #print(self.chords)
    
    def gen_rhythm(self):
        if len(self.rhythm_intro) == 0:
            for chord in self.chords:
                total = 0
                measure = []
                while total < 4:
                    # pick a weighted random note
                    r1 = random.uniform(0, 1)
                    #new_note = r1
                    for i in range(0, len(self.all_rhythm_weights)):
                        if self.all_rhythm_weights[i] <= r1:
                            new_note = self.all_rhythms[i]                            
                    
                    # only add note size that fits inside the measure
                    if total + new_note > 4:
                        new_note = 4 - total
                    measure.append(new_note)
                    total += new_note
                    
                    # some percent chance to replicate the same note, creating runs of the same size
                    # only if the note is < one beat long
                    if new_note < 1 and total + new_note <= 4:
                        r2 = random.uniform(0, 1)
                        if r2 < 0.5:
                            measure.append(new_note)
                            total += new_note
                    
                self.rhythm.append(measure)
                
        else:
            for i in range(0, len(self.chords)):
                num_notes = self.rhythm_intro[i]
                total = 4
                rough_notes = total / num_notes
                raw_measure = [rough_notes for i in range(0, num_notes)]
                measure = []
                while sum(measure) != 4:
                    measure = []
                    for note in raw_measure:
                        distances = [(note - n)**2 for n in self.all_rhythms]

                        min1 = distances.index(min(distances))
                        distances[min1] = 10000
                        min2 = distances.index(min(distances))
                        
                        measure.append(random.choice([self.all_rhythms[min1], self.all_rhythms[min2]]))
                    
                
                #print(measure)
                        
                self.rhythm.append(measure)
                        
                
    
    def gen_melody(self):
        for i in range(0, len(self.chords)):
            starting_note = random.choice(self.chord_notes[i]) 
            measure = [[starting_note, self.rhythm[i][0]]]
            for j in range(1, len(self.rhythm[i])):
                prev_note = measure[j-1][0]
                scale_i = self.scale.index(prev_note)
                if self.rhythm[i][j] < 2:
                    note = random.choice([self.scale[(scale_i + len(self.scale) - 1) % len(self.scale)], 
                                          self.scale[(scale_i + len(self.scale) + 1) % len(self.scale)]])
                else:
                    note = random.choice(self.chord_notes[i]) 
                measure.append([note, self.rhythm[i][j]])
            self.melody.append(measure)
    
    def gen_MIDI(self):
        #print(str(self.melody))
        # create your MIDI object
        mf = MIDIFile(2)     # only 1 track
        track1 = 0   # the only track
        track2 = 1

        time = 0    # start at the beginning
        mf.addTrackName(track1, time, self.file_name + "_chords")
        mf.addTrackName(track2, time, self.file_name + "_melody")
        #mf.addTempo(track1, time, 120)

        # add some notes
        channel = 0
        channel_chords = 0
        channel_melody = 1
        volume = 100
        
        overall_chord_time = 0
        for v in range(self.verses):
            for i in range(0, len(self.chord_notes)):
                chord = self.chord_notes[i]
                chord_time = overall_chord_time + i * 4
                pitch1 = self.notes_dict[chord[0]][0]
                pitch2 = self.notes_dict[chord[1]][0]
                pitch3 = self.notes_dict[chord[2]][0]
                duration = 4
                mf.addNote(track1, channel, pitch1, chord_time, duration, 65)
                mf.addNote(track1, channel, pitch2, chord_time, duration, 65)
                mf.addNote(track1, channel, pitch3, chord_time, duration, 65)
            overall_chord_time += len(self.chord_notes) * 4
        
        last_pitch = 0
        for i in range(0, len(self.melody)):
            measure = self.melody[i]
            measure_time = i * 4
            
            for j in range(0, len(measure)):
                note = measure[j][0]
                pitch1 = self.notes_dict[note][1]
                pitch2 = self.notes_dict[note][2]
                
                dist1 = (pitch1 - last_pitch)**2
                dist2 = (pitch2 - last_pitch)**2
                
                pitch = pitch1 if dist1 <= dist2 else pitch2                
                duration = measure[j][1]
                mf.addNote(track2, channel, pitch, measure_time, duration, volume)
                
                measure_time += measure[j][1]
                last_pitch = pitch
            

        # write it to disk
        with open("midi_files\\" + self.file_location + self.file_name + ".mid", 'wb') as outf:
            mf.writeFile(outf)

In [56]:
note_list = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
key = random.choice(note_list)

In [58]:
test_song = SongGeneration(key, length=4, verses=4, file_name="verse1", file_location="english_project\\")
test_song.gen_song()
print(test_song)

['B', 'C#', 'D#', 'E', 'F#', 'G#', 'A#']
['I', 'vi', 'IV', 'vii']



In [64]:
verse1 = SongGeneration(key, length=4, verses=4, file_name="verse1", file_location="english_project\\")
verse1.gen_song()

chorus = SongGeneration(key, length=4, verses=2, file_name="chorus", file_location="english_project\\")
chorus.gen_song()

verse2 = SongGeneration(key, length=4, verses=4, file_name="verse2", file_location="english_project\\")
verse2.gen_song()

bridge = SongGeneration(key, length=7, verses=1, file_name="bridge", file_location="english_project\\")
bridge.gen_song()

In [ ]:
now = str(datetime.now()) # current date and time
now = now.replace(":", "")
now = now.replace(" ", "_")
now = now.replace(".", "_")

### NLTK Stuff
---

In [ ]:
import nltk
from nltk.corpus import cmudict
from nltk.tokenize import RegexpTokenizer
import math
d = cmudict.dict()

In [ ]:
def nsyl(word):
    vowels = 'aeiouy'
    if len(word) == 1 and word.lower()[0] not in vowels:
        return [0]
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]]
    except KeyError:
        #if word not found in cmudict
        return [syllables(word)]

def syllables(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    return count

In [ ]:
f = open("elegaic_sonnets.txt")
raw_sonnets = f.readlines()
f.close()
print(raw_sonnets)

In [ ]:
sonnet = []
for line in raw_sonnets:
    new_line = line[:-1]
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(new_line)
    sonnet.append(tokens)
print(sonnet)

In [ ]:
syllable_list = []
for line in sonnet:
    new_line = []
    for word in line:
        new_line.append([word.lower(), nsyl(word)])
    syllable_list.append(new_line)
print(syllable_list)

In [ ]:
counts = []
lyrics = []
for line in syllable_list:
    count = 0
    sentence = []
    for word in line:
        #sentence += word[0] + " "
        count += max(word[1])
    counts.append(count)
    lyrics.append(sentence)
    
    next_count = 0
    sentence.append("")
    for word in line:
        next_count += max(word[1])
        if next_count > math.ceil(count/2) and len(sentence) == 1:
            sentence.append("")
        sentence[-1] += word[0] + " "
        
print(counts)
print(lyrics)

In [ ]:
counts_split = []
for i in range(0, len(counts)-1):
    count1 = counts[i]
    count2 = counts[i+1]
    count3 = [math.ceil(count1/2), math.floor(count1/2), math.ceil(count2/2), math.floor(count2/2)]
    counts_split.append(count3)
print(counts_split)

In [ ]:
stanza_num = 1
for stanza in counts_split:
    count_song = SongGeneration('G', length=len(stanza), rhythm=stanza, file_name="stanza_"+str(stanza_num))
    count_song.gen_song()
    print("Generated stanza " + str(stanza_num))
    stanza_num += 1